# Student Performance - Feature Engineering

## Set Up Environment

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
scores = pd.read_csv('StudentsPerformance.csv')
scores.head(3)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93


## Define Classes

In [ ]:
class Group_Metrics:
    def __init__(self, df):
        self.data = df
    
    def insert_group_metric(self, group, metric_col, metric_name, func):
        group_values = {}
        for g in self.data[group].unique():
            subset_df = self.slice_df({group: g})
            value = func(subset_df, metric_col)
            group_values[g] = value
        self.data['{} {} {}'.format(metric_col, group, metric_name)] = self.data[group].map(group_values)
    
    def slice_df(self, filters):
        cols = list(filters)
        vals = list(filters.values())
        subset_df = df
        for i in range(len(filters)):
            subset_df = subset_df[cols[i] == vals[i]]
        return subset_df
    
    def _calc_mean(self, df, col):
        return round(df[col].mean(), 1)
    
    def _calc_median(self, df, col):
        return df[col].median()
    
    def _calc_min(self, df, col):
        return df[col].min()
    
    def _calc_max(self, df, col):
        return df[col].max()